# SciAuth Tutorial

In this tutorial you will learn how to:
1. Create a SciToken signing key.
2. Publish the key to a website to create an issuer.
3. Use a token to submit to a HTCondor node.

### Requirements
- This notebook already has the python-scitokens package installed.  You can install it with `pip install scitokens`

This page is a special `bash` type page rather than Python.  Commands entered into this page are executed on the command line, not within a python interpreter.

## Creating a Key

We first will create a private and public key to sign tokens that we will create.

Create the private key with the `scitokens-admin-create-key` program.

In [ ]:
scitokens-admin-create-key --create-keys --pem-private > private.pem

Take a look at the newly created private key, `private.pem`.  You may want to **backup** the file since it's an important private key.  Feel free to download it from the file browser at the left.

The **PEM** format is one of many formats to store a private key.  SSH keys are also stored in this PEM format.  Later, we will see another format, **JWKS**.

In [ ]:
cat private.pem

Next we will create the public key, in **JWKS** format so that we can upload it to the website.  The **kid** is important for the rest of the tutorial.

In [ ]:
scitokens-admin-create-key --private-keyfile private.pem --jwks-public

In order for an issuer to work, a server needs to serve:
1. A metadata page for clients to find the keys (`/.well-known/openid-configuration`) 
2. The public keys

The metadata page is very short with only 2 attributes.

Steps to create the issuer:
1. Create the configuration file on the server running this notebook.  The server already has this file, it can be found with the command:

In [ ]:
cat /www/token-issuer/.well-known/openid-configuration

2. Now we will create the keys file, which is a [JWKS](https://datatracker.ietf.org/doc/html/rfc7517) formatted public key to verify our tokens.

In [ ]:
scitokens-admin-create-key --private-keyfile private.pem --jwks-public > /www/token-issuer/keys

Try downloading the metadata file from the localhost

In [ ]:
curl https://localhost:8443/token-issuer/.well-known/openid-configuration

In [ ]:
curl https://localhost:8443/token-issuer/keys

## Creating your first Token

Next, we will create your first token. Be sure to update the **kid** with yours from above.

In [ ]:
scitokens-admin-create-token \
--keyfile=private.pem \
--key_id kid \
--issuer https://localhost:8443/token-issuer/ \
scope="write:/ read:/" \
aud="localhost"

Check out your token on a website like https://jwt.io or https://demo.scitokens.org.  Both websites will warn you about an invalid signature because they don't use key discovery or can't reach localhost:8443 on your server.

## Testing the Token

Next, we will try some authorizations with the tokens.  For example, we will test if a token can access specific paths.  We will use the command `scitokens-test-access` which comes with the scitokens-cpp package.

First, we will create a new token with a restricted scope of `read:/protected`

In [ ]:
scitokens-admin-create-token \
--keyfile=private.pem \
--key_id kid \
--issuer https://localhost:8443/token-issuer/ \
scope="read:/protected" \
aud="localhost"

Next, we will run the command to test access to `/protected`.  The command we will use below, be sure to replace the **TOKEN** with the output of the above tommand.

In [ ]:
scitokens-test-access \
TOKEN \
https://localhost:8443/token-issuer/ \
localhost \
read /protected

Next, we will try a command that we expect not to work.  Lets see if the token allows us to read `/secret`

In [ ]:
scitokens-test-access \
TOKEN \
https://localhost:8443/token-issuer/ \
localhost \
read /secret

The command failed!  The output says that it failed when trying to verify the `scope` claim, which we set above to only allow reading of `/protected`, not `/secret`.

## Submitting your first job

Lets submit our first job.  It will use the token you just created.  You will be submitting to HTCondor on the notebook server that is already configured to accept the issuer https://localhost:8443/token-issuer/.

### Background of HTCondor

HTCondor is a job scheduler for high throughput computing.  A user can submit jobs to it HTCondor which will manage the execution as well as the data movement.

First, lets try issuing a `condor_q` without any tokens in our environment.  It should **fail**!

In [ ]:
condor_q

Lets copy our token to a known location. The `key_id` needs to match the `kid` from your public key you created above.

In [ ]:
scitokens-admin-create-token --keyfile=private.pem \
--key_id kid  \
--issuer https://localhost:8443/token-issuer \
aud="ANY" \
ver="scitokens:2.0" \
sub="jovyan" \
scope="condor:/READ condor:/WRITE" > token

Save the token location to an environment variable for HTCondor to find.

In [ ]:
export BEARER_TOKEN_FILE=$PWD/token

In [ ]:
condor_q

Now, lets submit a job.
The HTCondor schedd is on a remote machine, so we have to use the `-spool`
command to send the input files to the remote machine.

In [ ]:
condor_submit job.txt

It may take some time for the job to schedule and run.  Feel free to move on and come back to check on it.
Remember, the token will expire and you will need to renew it occasionally.

In [ ]:
condor_q

In [ ]:
cat log

## Get a token from an issuer

We will be using [oidc-agent](https://indigo-dc.gitbook.io/oidc-agent/) to download a token to also submit a job.  This time, the issuer will instead by a public issuer, not a private key we generated.

**Since oidc-agent uses a lot of interaction, we will be using the jupyter terminal. To open a terminal, Go to File -> New -> Terminal**

The first step is to start the oidc-agent process.  It will ask you several questions.  For scopes, select max.  You can just hit "enter" through the encryption password questions.

In the **terminal**, run these two commands.  After they have completed successfully, come back to this notebook.

```bash
eval `oidc-agent-service use` && oidc-gen issuer --issuer=https://lw-issuer.osgdev.chtc.io/scitokens-server/
```

Connect to the service in this notebook.

In [ ]:
eval `oidc-agent-service use`

Get the token and put it on the command line with the `oidc-token` command:

In [ ]:
oidc-token issuer

Again, check the token in [jwt.io](jwt.io)

Save the new token to the `token` file.

In [ ]:
oidc-token issuer > token

In [ ]:
condor_q

In [ ]:
cat log

# Recap

1. Created a public and private key for a token.
2. Installed the public key on a webserver in order for token to be validated against it.
3. Created a token and tested some of the authorizations
4. Used the token to submit a job to condor
5. We used oidc-agent to go through a OAuth flow to get a token from an external issuer.